In this notebook, your task is to explore a dataset of polysomnography data (PSG) from patients in a sleep clinic. The data is taken from a [PhysioNet](https://www.physionet.org/) repository called [Haaglanden Medisch Centrum Sleep Staging Database](https://physionet.org/content/hmc-sleep-staging/1.1/) by Diego Alvarez-Estevez and Roselyne Rijsman.

## Important: Run this code cell each time you start a new session!

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install os
!pip install mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import mne

In [ ]:
subject_list = [f'SN{str(i).zfill(3)}' for i in range(1, 4)]
fs = 256
old_base_folder = os.path.join('physionet.org', 'files', 'hmc-sleep-staging', '1.1', 'recordings')

In [ ]:
for subject in subject_list:
    !wget -rNcnp https://physionet.org/files/hmc-sleep-staging/1.1/recordings/{subject}.edf
    !wget -rNcnp https://physionet.org/files/hmc-sleep-staging/1.1/recordings/{subject}_sleepscoring.edf

In [ ]:
# Make a simple directory to hold the data
if not os.path.exists('data'):
    os.makedirs('data')

# Iterate through the filenames
for subject in subject_list:
    # Load the annotations
    annotations = mne.read_annotations(os.path.join(old_base_folder, f'{subject}_sleepscoring.edf'))
    annotations_data = {
        'onset': annotations.onset,
        'duration': annotations.duration,
        'description': annotations.description
    }

    # Load the data
    edf = mne.io.read_raw_edf(os.path.join(old_base_folder, f'{subject}.edf'))
    df = pd.DataFrame(edf.get_data().T, columns=edf.ch_names)
    df['Time'] = np.arange(0, len(df))*(1/fs)

    # Add the annotations as a new column
    df['Annotation'] = np.nan
    annotations_data = zip(annotations.onset,
                           annotations.duration,
                           annotations.description)
    for onset, duration, description in annotations_data:
        # Skip if annotation relates to lights
        if 'Light' in description:
            continue

        # Add annotation to df
        label = description.split(' ')[-1]
        window_condition = (df['Time'] >= onset) & (df['Time'] < onset+duration)
        df.loc[window_condition, 'Annotation'] = label

    # Clean up the table save
    first_cols = ['Time', 'Annotation']
    new_col_order = first_cols + [col for col in df.columns if col not in first_cols]
    df = df[new_col_order]
    df.dropna(inplace=True)
    df.to_csv(os.path.join('data', f'{subject}.csv'), index=False)

In [ ]:
df

# Instructions

To get full credit for this assignment, we should be able to run your entire notbook from start to finish without any errors. You can check this yourself by selecting "Runtime" > "Run all" in the Google Colab menu.

# Overview of the Dataset

There are several stages of sleep that occur in a cycle throughout the night. For the sake of this dataset, sleep can be divided into the following stages: awake (W), light sleep (N1), intermediate sleep (N2), deep sleep (N3), and REM sleep (R). Each stage of sleep plays an important role in the overall health and well-being of an individual. Therefore, sleep stage classification is a useful technique for diagnosing sleep disorders.

The goal of this dataset is to enable researchers to automatically identify sleep stages using polysomnographic (PSG) recordings. Given the large size of these recordings, we will work only work with data from 3 participants from this 151-person study. Patient recordings were randomly selected from a heterogeneous group of patients who were referred for PSG examination due to different sleep disorders.

Participants were instrumented with the minimal recommended set of sensors necessary to visually score sleep stages: four EEG (F4/M1, C4/M1, O2/M1, and C3/M2), two EOG (E1/M2 and E2/M2), one bipolar chin EMG, and one ECG (single modified lead II) data stream. Participants slept in the sleep clinic for one night. Their PSG recordings were reviewed and annotated by expert clinicians according to discrete 30-second windows. An example of this annotation process is shown below (note: this is merely an illustration and does not represent the true time-scale of the data):

<img src="https://drive.google.com/uc?id=1UnkMe1WmP813ZGyS7ynT_CMqP2Nr2do0" width=750px/>

All data was recorded at 256 Hz. For your convenience, the data has already been processed so that there is one `.csv` per participant in a folder called `data` with the following naming convention: `SN{subject_id}.csv`. These files have the following columns:

| Column # | Description |
|----------|-------------|
| 1 | Time in seconds|
| 2 | Annotation given by the sleep experts (W, N1, N2, N3, R) |
| 3–6 | EEG data |
| 7 | EMG data |
| 8–9 | EOG data |
| 10 | ECG data |

In [ ]:
# The relevant folders and files associated with this dataset
base_folder = 'data'

In this notebook, your goal will be to summarize the sensor data within each 30-second window in a way that makes it easy to discriminate between epochs during different sleep stages.

# Inspecting the Dataset

For this part of the notebook, your task is to complete some helper functions that will help you load and examine the dataset. An example `DataFrame` and some commands for testing these functions have been provided for you.

In [ ]:
df = pd.read_csv(os.path.join(base_folder, f'{subject_list[0]}.csv'))

**(Part 1)** Complete the function below so that it grabs a single 30-second window of data from a DataFrame `df` starting from `start_time`. The function should also return the annotation associated with that window.

In [ ]:
def grab_window(df, start_time):
    """
    Grab a 30-second window of data from start_time to start_time+30
    df: the DataFrame
    start_time: the start of the window in seconds (must be a multiple of 30)
    """
    # Check the inputs
    if start_time % 30 != 0:
        raise Exception('start_time should be a multiple of 30')

    # TODO: Write code here to get the data window of interest
    window_df = None

    # TODO: Write code here to get the annotation associated with the window
    annotation = None
    return window_df, annotation

In [ ]:
window_df, annotation = grab_window(df, 0)
print(f'Annotation: {annotation}')
window_df

In [ ]:
window_df, annotation = grab_window(df, 30*100)
print(f'Annotation: {annotation}')
window_df

**(Part 2)** Complete the function below so that it takes in a 30-second window of data and displays the data from a single column `sensor`as both as a time-series signal (signal over time) and as a spectrogram (frequency over time heatmap).

In [ ]:
from numpy.fft import fftfreq
from scipy.fftpack import fft
from scipy import signal
def view_recording(window_df, sensor):
    """
    Displays data in the sensor column as a time-domain signal and as a spectrogram
    window_df: the 30-second DataFrame
    sensor: the name of the sensor column to be displayed
    """
    # Check the inputs
    window_length = window_df['Time'].max()-window_df['Time'].min()
    if window_length > 30:
        raise Exception('Please only provide a 30-second window')
    if sensor not in list(df.columns):
        raise Exception('Sensor not found')

    # TODO: Write code here to plot the signal as a time-domain signal and as a spectrogram

In [ ]:
window_df, _ = grab_window(df, 0)
view_recording(window_df, 'EEG F4-M1')
view_recording(window_df, 'ECG')

In [ ]:
window_df, _ = grab_window(df, 30*100)
view_recording(window_df, 'EEG F4-M1')
view_recording(window_df, 'ECG')

# Extracting Information from a Recording

For this part of the notebook, your task is to create functions that will process the sensor data within a given 30-second window into a compact representation that will make it easier for us to compare windows across different sleep stages. It is up to you to decide the kind of information you want to extract from each window, but here are some pieces of advice:
* Examine the data using the `view_recording()` function you created earlier to see if you notice any interesting trends in the data.
* If you don't know where to start, consider using basic time-domain and frequency-domain metrics that we discussed in class.
* It is okay to use the same metrics for each sensor.

You may use the following window to test your functions:

In [ ]:
window_df, _ = grab_window(df, 30*100)

Each function should return a `dict` of the following form:
```
{'metric name 1': 0.0000,
'metric name 2': 0.0000,
...}
```

**(Part 1)** Complete the function below to summarize the four columns of EEG data within a 30-second window.

In [ ]:
def compute_eeg_metrics(window_df, fs=256):
    """
    Returns a dictionary of metrics that summarize the EEG data in the DataFrame
    df: the DataFrame
    fs: the sampling rate
    """
    # TODO: Write code here to fill in a dictionary of metrics
    info_dict = None
    return info_dict

In [ ]:
compute_eeg_metrics(window_df)

**(Part 2)** Complete the function below to summarize the single column of EMG data within a 30-second window.

In [ ]:
def compute_emg_metrics(window_df, fs=256):
    """
    Returns a dictionary of metrics that summarize the EMG data in the DataFrame
    df: the DataFrame
    fs: the sampling rate
    """
    # TODO: Write code here to fill in a dictionary of metrics
    info_dict = None
    return info_dict

In [ ]:
compute_emg_metrics(window_df)

**(Part 3)** Complete the function below to summarize the two columns of EOG data within a 30-second window.

In [ ]:
def compute_eog_metrics(window_df, fs=256):
    """
    Returns a dictionary of metrics that summarize the EOG data in the DataFrame
    df: the DataFrame
    fs: the sampling rate
    """
    # TODO: Write code here to fill in a dictionary of metrics
    info_dict = None
    return info_dict

In [ ]:
compute_eog_metrics(window_df)

**(Part 4)** Complete the function below to summarize the single column of ECG data within a 30-second window.

In [ ]:
def compute_ecg_metrics(window_df, fs=256):
    """
    Returns a dictionary of metrics that summarize the ECG data in the DataFrame
    df: the DataFrame
    fs: the sampling rate
    """
    # TODO: Write code here to fill in a dictionary of metrics
    info_dict = None
    return info_dict

In [ ]:
compute_ecg_metrics(window_df)

**(Part 5)** If all of the functions above are complete, then you should be able to run the following function below to produce a single `dict` that includes the metrics across all sensors for a given `DataFrame`:

In [ ]:
def process_window(window_df):
    """
    Process a DataFrame and produce all of the metrics as a dictionary
    (one value per key)
    window_df: the DataFrame
    """
    # Extract metrics
    eeg_dict = compute_eeg_metrics(window_df)
    emg_dict = compute_emg_metrics(window_df)
    eog_dict = compute_eog_metrics(window_df)
    ecg_dict = compute_ecg_metrics(window_df)

    # Combine everything into a single dictionary
    final_dict = {}
    final_dict.update(eeg_dict)
    final_dict.update(emg_dict)
    final_dict.update(eog_dict)
    final_dict.update(ecg_dict)

    # Check that the keys are unique from the original metrics
    if len(final_dict) != len(eeg_dict) + len(emg_dict) + len(eog_dict) + len(ecg_dict):
        raise Exception('Make sure the keys of your dictionaries are unique')
    return final_dict

In [ ]:
process_window(window_df)

# Creating Our Processed Dataset

For this part of the notebook, simply run the code below to create your final dataset. This will take a long time because the recordings are long. When the code is done running, notice how each window corresponds to a single row.

In [ ]:
# Iterate through the subject IDs
df = pd.DataFrame()
for subject in subject_list:
    # Load the DataFrame
    subject_df = pd.read_csv(os.path.join(base_folder, f'{subject}.csv'))

    # Iterate through all windows
    window_starts = np.arange(0, subject_df['Time'].max(), 30)
    for window_start in window_starts:
        # Grab the window
        window_df, annotation = grab_window(subject_df, window_start)

        # Process the window
        result_dict = process_window(window_df)

        # Add it to the final DataFrame
        result_dict['Subject ID'] = subject
        result_dict['Start Time'] = window_start
        result_dict['Annotation'] = annotation

        result_df = pd.DataFrame([result_dict])
        df = pd.concat([df, result_df], axis=0)

In [ ]:
first_cols = ['Subject ID', 'Start Time', 'Annotation']
new_col_order = first_cols + [col for col in df.columns if col not in first_cols]
df = df[new_col_order]
df

# Exploring Our PSG Data Characteristics

For this part of the notebook, your task is to use the techniques we discussed in class to the various techniques we discussed in class (e.g., descriptive statistics, histograms) to investigate whether the window characteristics you calculated are useful for discriminating different sleep stages.

You do not need to look at every single metric using every analysis technique. However, it is important that you are able to show that you come away from this homework feeling comfortable that at least a few of your metrics are discriminative since you will use this dataset in later assignments. If none of your metrics are discriminative, then you should consider extracting different metrics from the windows.

For your convenience, the code below retrieves the names of all of your metrics:

In [ ]:
metrics = list(filter(lambda x: x not in first_cols, df.columns))
metrics

**(Part 1)** Calculate the average and standard deviation of your metrics within each sleep stage. For the sake of readability, you do not need to show these statistics for every single metric.

In [ ]:
# Write code here

**(Part 2)** Generate histograms showing how the distribution of your metrics differ across sleep stages. For the sake of readability, you do not need to show these statistics for every single metric.

In [ ]:
# Write code here

**(Part 3)** The following code has already been written for your to conduct statistical tests comparing your metrics across sleep stages. Notice that the code uses different statistical tests than the ones we did in class: ANOVA instead of a t-test and a Kruskal-Wallis instead of a Mann-Whitney U test. This is because we have more than two categories that we are interested in comparing.

Run this code to determine whether or not any of your metrics are significantly different across sleep stages.

In [ ]:
from scipy import stats

def compare_distributions(df, col):
    # Get the relevant data
    data = df[col]
    df_w_data = df[df['Annotation'] == 'W'][col]
    df_n1_data = df[df['Annotation'] == 'N1'][col]
    df_n2_data = df[df['Annotation'] == 'N2'][col]
    df_n3_data = df[df['Annotation'] == 'N3'][col]
    df_r_data = df[df['Annotation'] == 'R'][col]

    # Check of the data is normally distributed
    statistic, p_value = stats.normaltest(data)
    test = None
    if p_value > 0.05:
        # Data is normally distributed, use ANOVA
        test = 'ANOVA'
        statistic, p_value = stats.f_oneway(df_w_data, df_n1_data, df_n2_data, df_n3_data, df_r_data)
    else:
        # Data is not normally distributed, use Kruskal-Wallis Test
        test = 'Kruskal-Wallis'
        statistic, p_value = stats.kruskal(df_w_data, df_n1_data, df_n2_data, df_n3_data, df_r_data)

    print(f'Result of {test} for {col}: {statistic}, p-value is {p_value:0.3f}')

In [ ]:
for col in metrics[:5]:
    compare_distributions(df, col)

# Bookkeeping

**IMPORTANT:** Once you are happy with your `DataFrame`, export it as a `.csv` and save it somewhere on your local machine. We will use this `.csv` in future assignments.

In [ ]:
df.to_csv('psg_data.csv', index=False)